# ipython -c "%run ProjectPart1.ipynb"  ..RUN CODE WITH THIS..

## Import Needed Packages


In [1]:
# import the necessary packages
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sys
import io
from sklearn.metrics import classification_report

from keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras import applications
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, RMSprop
from tensorflow.keras.initializers import RandomNormal, RandomUniform
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalHinge
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
import tensorflow as tf
from scipy import ndimage

import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
from matplotlib.backends.backend_agg import FigureCanvasAgg
import numpy as np
import argparse
import os
import random
import shutil

from os import listdir

from keras.preprocessing.image import ImageDataGenerator

## Load Dataset

In [19]:

ORIG_INPUT_DATASET = "Dataset"
BASE_PATH = "DatasetNew"

# Derive the training, validation, and testing directories
TRAIN_PATH = os.path.join(BASE_PATH, "training")
VAL_PATH = os.path.join(BASE_PATH, "validation")
TEST_PATH = os.path.join(BASE_PATH, "testing")

# Define the amount of data that will be used for training
TRAIN_SPLIT = 0.8

# The amount of validation data will be a percentage of the
# training data
VAL_SPLIT = 0.1

GPU available: []


Comment This when you Test !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! but when you want to train you need this and make newDataset  empty

In [ ]:
# grab the paths to all input images in the original input directory
# and shuffle them

imagePaths = list(paths.list_images(ORIG_INPUT_DATASET))
random.seed(42)
random.shuffle(imagePaths)
# compute the training and testing split
i = int(len(imagePaths) * TRAIN_SPLIT)
trainPaths = imagePaths[:i]
testPaths = imagePaths[i:]
# we'll be using part of the training data for validation
i = int(len(trainPaths) * VAL_SPLIT)
valPaths = trainPaths[:i]
trainPaths = trainPaths[i:]
# define the datasets that we'll be building
datasets = [
	("training", trainPaths, TRAIN_PATH),
	("validation", valPaths, VAL_PATH),
	("testing", testPaths, TEST_PATH)
]

Comment This when you Test !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! but when you want to train you need this and make newDataset  empty

## loop over the datasets

In [ ]:

for (dType, imagePaths, baseOutput) in datasets:
	# show which data split we are creating
	print("[INFO] building '{}' split".format(dType))
	# if the output base output directory does not exist, create it
	if not os.path.exists(baseOutput):
		print("[INFO] 'creating {}' directory".format(baseOutput))
		os.makedirs(baseOutput)
	# loop over the input image paths
	for inputPath in imagePaths:
		# extract the filename of the input image and extract the
		# class label ("0" for "negative" and "1" for "positive")
		filename = inputPath.split(os.path.sep)[-1]
		label = filename[-5:-4]
		# build the path to the label directory
		labelPath = os.path.sep.join([baseOutput, label])
		# if the label output directory does not exist, create it
		if not os.path.exists(labelPath):
			print("[INFO] 'creating {}' directory".format(labelPath))
			os.makedirs(labelPath)
		# construct the path to the destination image and then copy
		# the image itself
		p = os.path.sep.join([labelPath, filename])
		shutil.copy2(inputPath, p)# Define Model
 

Original Model

In [ ]:

class CancerNet:
  @staticmethod
  def build(width, height, depth, classes):
    height, width, depth = 50, 50, 3  # Assuming 3 channels for RGB images
    inputShape = (height, width, depth)
    chanDim = -1
    if K.image_data_format() == "channels_first":
      inputShape = (depth, height, width)
      chanDim = 1
    model = Sequential()
    # CONV => RELU => POOL
    model.add(SeparableConv2D(32, (3, 3), padding="same", input_shape = inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # (CONV => RELU => POOL) * 2
    model.add(SeparableConv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(SeparableConv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # (CONV => RELU => POOL) * 3
    model.add(SeparableConv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(SeparableConv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(SeparableConv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # first (and only) set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    # softmax classifier
    model.add(Dense(classes))
    model.add(Activation("softmax"))
    # return the constructed network architecture
    return model

Customized model

In [1]:
# class CancerNet:
#   @staticmethod
#   def build(width, height, depth, classes):
#     height, width, depth = 50, 50, 3  # Assuming 3 channels for RGB images
#     inputShape = (height, width, depth)
#     chanDim = -1
#     if K.image_data_format() == "channels_first":
#       inputShape = (depth, height, width)
#       chanDim = 1
#     model = Sequential()
#     # CONV => RELU => POOL
#     model.add(SeparableConv2D(16, (3, 3), padding="same", input_shape = inputShape))
#     model.add(Activation("relu"))
#     model.add(BatchNormalization(axis=chanDim))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
#     # (CONV => RELU => POOL) * 2
#     model.add(SeparableConv2D(32, (3, 3), padding="same"))
#     model.add(Activation("relu"))
#     model.add(BatchNormalization(axis=chanDim))
#     model.add(SeparableConv2D(32, (3, 3), padding="same"))
#     model.add(Activation("relu"))
#     model.add(BatchNormalization(axis=chanDim))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
#     # first (and only) set of FC => RELU layers
#     print(model.output_shape)
#     print("??????????????????????????")
#     model.add(Flatten())
#     print(model.output_shape)
#     model.add(Dense(256))
#     print(model.output_shape)
#     model.add(Activation("relu"))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.5))
#     # softmax classifier
#     model.add(Dense(classes))
#     model.add(Activation("softmax"))
#     # return the constructed network architecture
#     return model

# Attention Layer

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import SeparableConv2D, Activation, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense

# class SelfAttentionLayer(tf.keras.layers.Layer):
#     def __init__(self, filters):
#         super(SelfAttentionLayer, self).__init__()
#         self.filters = filters
#         self.gamma = self.add_weight(name="gamma", shape=(1,), initializer="zeros", trainable=True)

#     def build(self, input_shape):
#         self.convolution_f = SeparableConv2D(filters=self.filters, kernel_size=(1, 1), strides=(1, 1), padding='same')
#         self.convolution_g = SeparableConv2D(filters=self.filters, kernel_size=(1, 1), strides=(1, 1), padding='same')
#         self.convolution_h = SeparableConv2D(filters=self.filters, kernel_size=(1, 1), strides=(1, 1), padding='same')
#         super(SelfAttentionLayer, self).build(input_shape)

#     def call(self, x):
#         f = self.convolution_f(x)
#         g = self.convolution_g(x)
#         h = self.convolution_h(x)

#         s = tf.matmul(g, f, transpose_b=True)
#         beta = tf.nn.softmax(s, axis=-1)
#         o = tf.matmul(beta, h)

#         x = self.gamma * o + x

#         return x
    
# class CancerNet:
#     @staticmethod
#     def build(width, height, depth, classes):
#         height, width, depth = 50, 50, 3  # Assuming 3 channels for RGB images
#         inputShape = (height, width, depth)
#         chanDim = -1

#         model = Sequential()

#         # CONV => RELU => POOL
#         model.add(SeparableConv2D(32, (3, 3), padding="same", input_shape=inputShape))
#         model.add(Activation("relu"))
#         model.add(BatchNormalization(axis=chanDim))
#         model.add(MaxPooling2D(pool_size=(2, 2)))
#         model.add(Dropout(0.25))

#         # Attention Layer
#         model.add(SelfAttentionLayer(filters=32))

#         # (CONV => RELU => POOL) * 2
#         model.add(SeparableConv2D(64, (3, 3), padding="same"))
#         model.add(Activation("relu"))
#         model.add(BatchNormalization(axis=chanDim))
#         model.add(SeparableConv2D(64, (3, 3), padding="same"))
#         model.add(Activation("relu"))
#         model.add(BatchNormalization(axis=chanDim))
#         model.add(MaxPooling2D(pool_size=(2, 2)))
#         model.add(Dropout(0.25))

#         # Attention Layer
#         model.add(SelfAttentionLayer(filters=64))

#         # (CONV => RELU => POOL) * 3
#         model.add(SeparableConv2D(128, (3, 3), padding="same"))
#         model.add(Activation("relu"))
#         model.add(BatchNormalization(axis=chanDim))
#         model.add(SeparableConv2D(128, (3, 3), padding="same"))
#         model.add(Activation("relu"))
#         model.add(BatchNormalization(axis=chanDim))
#         model.add(SeparableConv2D(128, (3, 3), padding="same"))
#         model.add(Activation("relu"))
#         model.add(BatchNormalization(axis=chanDim))
#         model.add(MaxPooling2D(pool_size=(2, 2)))
#         model.add(Dropout(0.25))

#         # Attention Layer
#         model.add(SelfAttentionLayer(filters=128))

#         # First (and only) set of FC => RELU layers
#         model.add(Flatten())
#         model.add(Dense(256))
#         model.add(Activation("relu"))
#         model.add(BatchNormalization())
#         model.add(Dropout(0.5))

#         # Softmax classifier
#         model.add(Dense(classes))
#         model.add(Activation("softmax"))

#         # Return the constructed network architecture
#         return model

Comment This when you Test !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! but when you want to train you need this and make newDataset  empty

Using different model

In [ ]:

# class CancerNet:
#     @staticmethod
#     def VGG(pretrained = True, input_shape = (50, 50, 3)):

#         if(pretrained):

#             base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape, pooling='none')
#         else:
#             base_model = applications.resnet50.ResNet50(weights=None, include_top=False, input_shape=input_shape, pooling='none')


#         return base_model


Freezing Def

In [ ]:
def freeze(model, number_of_frozen_layers):
    layers = model.layers

    layers = layers[:number_of_frozen_layers]

    for layer in layers:
        layer.trainable = False

    return model

In [ ]:
# initialize our number of epochs, initial learning rate, and batch
# size
NUM_EPOCHS = 40
# NUM_EPOCHS = 300
# NUM_EPOCHS = 1000
INIT_LR = 1e-2
BS = 32
# determine the total number of image paths in training, validation,
# and testing directories
trainPaths = list(paths.list_images(TRAIN_PATH))
totalTrain = len(trainPaths)
totalVal = len(list(paths.list_images(VAL_PATH)))
totalTest = len(list(paths.list_images(TEST_PATH)))
# calculate the total number of training images in each class and
# initialize a dictionary to store the class weights
trainLabels = [int(p.split(os.path.sep)[-2]) for p in trainPaths]
trainLabels = to_categorical(trainLabels)
classTotals = trainLabels.sum(axis=0)
classWeight = dict()
# loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
	classWeight[i] = classTotals.max() / classTotals[i]

Data Augmentation

In [ ]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rescale=1 / 255.0,
	rotation_range=20,
	zoom_range=0.05,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.05,
	horizontal_flip=True,
	vertical_flip=True,
	fill_mode="nearest")
# initialize the validation (and testing) data augmentation object
valAug = ImageDataGenerator(rescale=1 / 255.0)

Changin in zoom range and rotation range

In [ ]:
# # initialize the training data augmentation object
# trainAug = ImageDataGenerator(
# 	rescale=1 / 255.0,
# 	rotation_range=10,
# 	zoom_range=0.5,
# 	width_shift_range=0.1,
# 	height_shift_range=0.1,
# 	shear_range=0.05,
# 	horizontal_flip=True,
# 	vertical_flip=True,
# 	fill_mode="nearest")
# # initialize the validation (and testing) data augmentation object
# valAug = ImageDataGenerator(rescale=1 / 255.0)

Without rotation and fill mode changed to 

In [ ]:
# # initialize the training data augmentation object
# trainAug = ImageDataGenerator(
# 	rescale=1 / 255.0,
# 	zoom_range=0.5,
# 	width_shift_range=0.1,
# 	height_shift_range=0.1,
# 	shear_range=0.05,
# 	horizontal_flip=False,
# 	vertical_flip=False,
# 	fill_mode="reflect")
# # initialize the validation (and testing) data augmentation object
# valAug = ImageDataGenerator(rescale=1 / 255.0)

No augmentations

In [ ]:
# # initialize the training data augmentation object
# trainAug = ImageDataGenerator()
# # initialize the validation (and testing) data augmentation object
# valAug = ImageDataGenerator()

- Training generator
- Validation generator
- Testing generator

In [ ]:
# initialize the training generator
trainGen = trainAug.flow_from_directory(
	TRAIN_PATH,
	class_mode="categorical",
	target_size=(48, 48),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)
# initialize the validation generator
valGen = valAug.flow_from_directory(
	VAL_PATH,
	class_mode="categorical",
	target_size=(48, 48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)
# initialize the testing generator
testGen = valAug.flow_from_directory(
	TEST_PATH,
	class_mode="categorical",
	target_size=(48, 48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

# Train Model

In [ ]:
# initialize our CancerNet model and compile it

model = CancerNet.build(width=48, height=48, depth=3,
	classes=2)   # This is using original model


# model = CancerNet.Res50(True,(50,50,3))   # This is using VGG model (as a test) <<<<<Not Working>>>

# model = freeze(model, 5) # Freezing the model - new!


# opt = Adagrad(learning_rate=INIT_LR, decay=INIT_LR / NUM_EPOCHS) # Using Adagrad  <<<< Base Test

opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / NUM_EPOCHS) # Using Adam


# Define callbacks
# checkpoint = ModelCheckpoint('model_checkpoint.h5', save_best_only=True)
# csv_logger = CSVLogger('training_history.csv', append=True)


model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# fit the model
H = model.fit(
	x=trainGen,
	steps_per_epoch=totalTrain // BS,
	validation_data=valGen,
	validation_steps=totalVal // BS,
	class_weight=classWeight,
	epochs=NUM_EPOCHS)



# H = model.fit(
# 	x=trainGen,
# 	steps_per_epoch=totalTrain // BS,
# 	validation_data=valGen,
# 	validation_steps=totalVal // BS,
# 	class_weight=classWeight,
# 	epochs=NUM_EPOCHS,
#     callbacks=[checkpoint, csv_logger])


# H = model.fit(
# 	x=trainGen,
# 	validation_data=valGen,
# 	class_weight=classWeight,    # Trying without controling steps per epoch
# 	epochs=NUM_EPOCHS)

In [ ]:
# Create a file to save the output


# reset the testing generator and then use our trained model to
# make predictions on the data
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict(x=testGen, steps=(totalTest // BS) + 1)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))

# Create a StringIO object to capture the output
output_buffer = io.StringIO()

# Replace the default stdout with the StringIO object
sys.stdout = output_buffer

# Your code block here
print("[INFO] evaluating network...")

# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))
# ...

# Get the captured output and write it to a file
output_text = output_buffer.getvalue()
with open("network_evaluation.txt", "w") as output_file:
    output_file.write(output_text)

# Reset stdout to its original state (console)
sys.stdout = sys.__stdout__

# Print Plot and Measures

Compute Accuracy, Sensitivity, Specificity

In [ ]:
# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(testGen.classes, predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))



# Create a StringIO object to capture the output
output_buffer = io.StringIO()

# Replace the default stdout with the StringIO object
sys.stdout = output_buffer


# show a nicely formatted classification report
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))
# ...

# Get the captured output and write it to a file
output_text = output_buffer.getvalue()
with open("Accuracy_Sensitivity_Specificity.txt", "w") as output_file:
    output_file.write(output_text)

# Reset stdout to its original state (console)
sys.stdout = sys.__stdout__

Plot

In [ ]:


# Your plot code
%matplotlib inline
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

# Create a FigureCanvasAgg object to render the figure
canvas = FigureCanvasAgg(plt.gcf())
canvas.draw()

# Save the rendered figure to a file
canvas.print_figure("plot.png", dpi=100, format="png")


In [ ]:
# SHAP-related code
import shap
import numpy as np

# Generate background samples using the training generator
background = next(trainGen)[0]

# Instantiate the deep explainer with the best CNN model and the background samples
e = shap.DeepExplainer(model, background)

# Obtaining the SHAP values on the first 10 images
shap_values = e.shap_values(valGen.next()[0][1:11])

# plot to see where did the model concentrate to classify the images
shap.image_plot(shap_values, -valGen.next()[0][1:11])